# Workathon Price Prediction Hackathon


![Alt text](https://analyticsindiamag.com/wp-content/uploads/2021/03/New-Hackathon-For-Data-Scientists-%E2%80%94-Workation-Price-Prediction-Challenge.jpeg)






# Overview

* Dataset Description:

* Train.csv - 21000 rows x 15 columns (Includes Per Person Price Column as Target variable)
* Test.csv - 9000 rows x 14 columns (Doesn't include the Target Variable)
* Sample Submission.csv - Please check the Evaluation section for more details on how to generate a valid submission

* Attributes:

* Uniq Id - Unique Identifier per row sample
* Package Name - Name of the tour package
* Package Type - Type of the tour package
* Destination - A destination place
* Itinerary - complete itinerary
* Places Covered - covered places in the itinerary
* Travel Date - Date of travel
* Hotel Details - Details of the hotel stay
* Start City - Start place for the travel
* Airline - Flight details
* Flight Stops - Intermediate stops if any
* Meals - Inflight meals or services
* Sightseeing Places Covered - Itinerary details regarding sightseeing
* Cancellation Rules - Cancellation policy as per travel company
*Per Person Price - Price of the tour package per person (Target Column)
 
The new covid-era has provided a new way of living the work-life balance. We have seen a lot of different websites providing packages to work from different locations. From Kashmir to Kanyakumari, from Gujarat to Assam we have collected packages in and around India. It becomes really difficult to find the best place with all the amenities such as high-speed internet, a comfortable stay as well as within the budget. To solve the real-world problem of finding the best deals for a calm and enjoying workation trip. Workation is the best way to work at a remote location with a recreational and rejuvenating vacation for the team.

In this competition, one has to use the knowledge of machine learning, deep learning, and model building to predict the price per person for your next workstation trip. The data has more than 18000+ rows of different packages with the details like start location, hotel type, cost per person, destination, Itinerary, and many more.

In [ ]:
!pip install catboost


In [ ]:
!pip install transformers
!pip install -U sentence-transformers

## Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.style as style # for styling the graphs
#_______________________________________________________________________________________________________________
# style.available (to know the available list of styles)
style.use('ggplot') # chosen style
plt.rc('xtick',labelsize=13) # to globally set the tick size
plt.rc('ytick',labelsize=13) # to globally set the tick size
# To print multiple outputs together
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Change column display number during print
pd.set_option('display.max_columns', 500)
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')
# To display float with 2 decimal, avoid scientific printing
pd.options.display.float_format = '{:.2f}'.format
import seaborn as sns
import warnings
# To display float with 2 decimal, avoid scientific printing
pd.options.display.float_format = '{:.2f}'.format
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder

In [ ]:
#Mounting Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load the Data

In [ ]:
train = pd.read_csv('/content/drive/My Drive/Workathon/Train.csv',low_memory=False)
test=pd.read_csv('/content/drive/My Drive/Workathon/Test.csv',low_memory=False)

In [ ]:
test['Package Name'].value_counts()

Delhi-Agra-Jaipur-Udaipur Holiday                              17
Bali Special 4 Nights                                          17
In the Lap of Nature - Shimla and Chail Holiday from Delhi     15
Amazing Kerala                                                 15
Splendid North East - Pelling Special (Value added Holiday)    15
                                                               ..
Tirupati Pilgrimage by Road from Chennai                        1
A Corbett Wildlife Experience from Delhi                        1
Goa 2 Nights - Hotels & Transfers                               1
Stunning Singapore with Bintan                                  1
Desert Getaway to Jaisalmer and Jodhpur with Luxury Stays       1
Name: Package Name, Length: 2065, dtype: int64

## Creating Fetures from Date column

In [ ]:
train['Travel Date']=pd.to_datetime(train['Travel Date'])
train['Journey_Day']=train['Travel Date'].dt.day
train['Journey_month']=train['Travel Date'].dt.month

test['Travel Date']=pd.to_datetime(test['Travel Date'])
test['Journey_Day']=test['Travel Date'].dt.day
test['Journey_month']=test['Travel Date'].dt.month
train['Quarter']=train['Travel Date'].dt.quarter
test['Quarter']=test['Travel Date'].dt.quarter

## Label Encoding

In [ ]:
le=LabelEncoder()
train['Start City']=le.fit_transform(train['Start City'])
test['Start City']=le.transform(test['Start City'])


train['Cancellation Rules']=le.fit_transform(train['Cancellation Rules'])
test['Cancellation Rules']=le.transform(test['Cancellation Rules'])


In [ ]:
premium_dict={'Budget':0,'Standard':1,'Deluxe':2,'Luxury':3,'Premium':4}

train['Package Type']=train['Package Type'].map(premium_dict)
test['Package Type']=test['Package Type'].map(premium_dict)

## Feature Engineering

In [ ]:
import re
#sum(list(map(int,re.findall(r"(\d+)N",A))))

In [ ]:
def get_trip_duration(val):
  return sum(list(map(int,re.findall(r"(\d+)N",val))))

def get_hotel_ratings(val):
  return np.mean((list(map(float,re.findall(r":(\d+\.\d+)",val)))))

train['Trip_Duration']=train['Itinerary'].apply(lambda x:get_trip_duration(x))
test['Trip_Duration']=test['Itinerary'].apply(lambda x:get_trip_duration(x))


train['Hotel_Ratings']=train['Hotel Details'].apply(lambda x:get_hotel_ratings(x))
test['Hotel_Ratings']=test['Hotel Details'].apply(lambda x:get_hotel_ratings(x))


train['Places Covered']=train['Places Covered'].str.replace('|',",")
train['Hotel Details']=train['Hotel Details'].str.replace('|',",")

test['Places Covered']=test['Places Covered'].str.replace('|',",")
test['Hotel Details']=test['Hotel Details'].str.replace('|',",")

train['Airline']=train['Airline'].str.replace('|',",")
test['Airline']=test['Airline'].str.replace('|',",")

train['Total_Places_Covered']=train['Places Covered'].str.split(',').str.len()
train['Total_No_Hotel_Stay']=train['Hotel Details'].str.split(',').str.len()


test['Total_Places_Covered']=test['Places Covered'].str.split(',').str.len()
test['Total_No_Hotel_Stay']=test['Hotel Details'].str.split(',').str.len()

train['Total_No_Flight_Taken']=train['Airline'].str.split(',').str.len()
test['Total_No_Flight_Taken']=test['Airline'].str.split(',').str.len()

In [ ]:
train['Hotel Details'].value_counts(normalize=True)

Not Available                                                                                                                                                                      0.14
Novotel Goa Resort & Spa:4.1                                                                                                                                                       0.01
AMARA GRAND BAGA, GOA- MMT Holidays Special:                                                                                                                                       0.01
Little Mastiff Dharamshala - Unit Of Pong View Hotel:3.9                                                                                                                           0.00
Novotel Goa Resort & Spa:4.2                                                                                                                                                       0.00
                                                                                

## Vectorization of Text Data

In [ ]:
from sentence_transformers import SentenceTransformer
sentence_embedder_bert = SentenceTransformer('bert-large-nli-stsb-mean-tokens')

In [ ]:
sentence_embeddings_bert = sentence_embedder_bert.encode(train[['Package Name','Destination','Itinerary','Hotel Details','Airline','Sightseeing Places Covered']].values.tolist(),batch_size=128,show_progress_bar=True)
sentence_embeddings_bert

array([[-0.06347734,  0.6535153 ,  0.5390947 , ...,  0.79479223,
         0.46600664,  0.4843054 ],
       [ 0.20747003,  1.2463322 ,  1.0881994 , ...,  0.56939787,
         0.8270949 ,  0.10304745],
       [-0.82861763, -0.0277136 ,  0.7808695 , ..., -0.0124567 ,
        -0.42434546, -0.8270335 ],
       ...,
       [ 0.32804826,  0.03598508,  0.6918415 , ...,  0.7667735 ,
         0.62646526,  0.4046149 ],
       [-0.9125753 , -0.8631122 ,  0.44306642, ..., -0.19136104,
         0.68176734, -0.8006273 ],
       [ 0.08611299, -0.13923314,  0.34641552, ...,  0.7541319 ,
         0.7645264 ,  0.9039651 ]], dtype=float32)

In [ ]:
sentence_embeddings_bert_test = sentence_embedder_bert.encode(test[['Package Name','Destination','Itinerary','Hotel Details','Airline','Sightseeing Places Covered']].values.tolist(),batch_size=128,show_progress_bar=True)
sentence_embeddings_bert_test

array([[-0.13713859, -0.5662331 ,  0.3514722 , ..., -0.86564255,
        -0.73181427, -0.2887824 ],
       [-0.4015214 ,  0.9632254 ,  0.81955135, ...,  0.42628342,
         0.4773425 ,  0.6708819 ],
       [ 0.01351006,  0.42729276,  0.88834345, ...,  0.94483805,
         0.8416696 ,  1.1655798 ],
       ...,
       [-0.35485938,  0.58360696,  1.0704541 , ...,  0.21012196,
         0.8805464 ,  0.7646717 ],
       [-0.3005656 ,  0.21709986,  0.57764643, ...,  0.731203  ,
        -0.08467775, -0.60917443],
       [-0.12916432,  0.43507302,  1.3128698 , ...,  0.12736024,
         0.25814545,  0.19395517]], dtype=float32)

In [ ]:
bert_df=pd.DataFrame(sentence_embeddings_bert)
bert_df_test=pd.DataFrame(sentence_embeddings_bert_test)
new_train=pd.concat([bert_df,train['Package Type'],train['Hotel_Ratings'],train['Start City'],train['Cancellation Rules'],train['Flight Stops'],train['Meals'],train['Journey_Day'],train['Journey_month'],train['Trip_Duration'],train['Per Person Price']],1)
new_train

new_test=pd.concat([bert_df_test,test['Package Type'],test['Hotel_Ratings'],test['Start City'],test['Cancellation Rules'],test['Flight Stops'],test['Meals'],test['Journey_Day'],test['Journey_month'],test['Trip_Duration']],1)
new_test

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,Package Type,Hotel_Ratings,Start City,Cancellation Rules,Flight Stops,Meals,Journey_Day,Journey_month,Trip_Duration,Per Person Price
0,-0.06,0.65,0.54,-0.32,-1.34,0.31,-1.01,-0.12,-0.03,0.03,-0.05,-0.18,-0.12,-0.15,-0.33,0.73,-0.09,0.89,-0.66,0.81,-0.34,0.11,-0.06,0.56,0.46,0.50,0.48,1.23,0.91,-0.39,-0.14,-0.06,0.75,-0.27,-1.16,0.94,-0.62,0.04,-0.13,0.34,-1.15,-0.18,-0.42,-0.03,0.53,0.32,-0.14,0.91,0.31,1.14,-0.38,-0.17,-0.39,-0.69,0.12,-0.60,-0.54,0.97,0.69,0.41,-0.00,-0.35,-0.33,-0.37,0.08,0.98,-0.76,0.41,-0.83,0.35,0.19,0.67,-1.61,0.73,-0.53,0.21,-0.44,0.09,-0.11,0.17,0.77,0.23,-0.40,0.64,0.18,-0.09,1.05,0.50,1.24,-0.52,-0.38,0.49,0.11,-0.57,-0.73,-0.20,-0.42,-0.11,-0.20,0.45,-1.09,-0.45,0.41,0.02,0.34,-0.30,0.31,-0.40,0.31,0.78,0.45,-0.80,-1.02,-0.54,0.46,1.11,-0.41,-0.22,-0.82,0.77,-0.20,0.07,0.32,0.81,0.05,-1.41,0.10,-1.02,1.54,0.15,-0.36,0.19,0.41,-0.51,0.46,0.29,-0.00,0.63,-0.78,-1.40,0.28,0.29,-0.18,0.46,-0.72,1.10,-0.05,-0.16,-0.14,0.35,-0.65,-0.44,1.44,-0.16,0.07,-0.02,0.58,-0.28,0.62,0.07,0.60,0.41,-0.76,-0.73,0.43,-0.62,1.45,1.25,0.15,0.26,-0.61,1.15,-0.36,0.41,-0.30,0.64,-0.29,0.28,0.16,-0.98,-0.52,0.22,0.66,-0.02,0.53,-0.39,0.43,-0.83,0.45,-0.41,0.76,-0.89,-0.19,0.46,0.25,0.80,-0.40,-0.40,-0.54,-1.39,-0.43,0.92,-0.84,1.34,-0.59,-0.18,0.48,-0.02,-0.51,-0.25,-0.63,0.62,0.62,-0.82,0.29,-0.63,0.18,0.03,0.71,0.34,0.11,0.15,-0.44,0.15,-0.45,0.23,-0.63,0.08,-0.21,0.15,-0.70,-0.00,2.02,-0.75,-0.26,-0.47,-0.21,-0.04,-0.47,-0.92,0.21,-0.02,0.21,-0.17,0.19,0.41,0.56,0.12,0.40,0.54,...,0.37,1.17,-0.73,-0.45,-0.21,1.31,0.02,-1.10,0.09,0.18,-0.38,-0.12,-0.63,0.39,-0.34,0.00,-0.13,0.63,-0.17,0.85,0.49,0.03,-0.04,-0.43,-0.71,1.08,-0.06,0.24,0.14,0.58,0.14,-0.41,0.75,-0.06,-0.77,-0.20,0.81,0.02,-0.44,0.49,0.11,0.24,-0.11,-0.32,-0.28,-0.47,-0.13,0.67,-0.90,-0.17,0.33,-0.31,-1.10,-0.54,0.67,-0.48,-0.82,0.00,0.45,0.18,0.40,0.75,0.45,-0.23,0.34,0.10,-0.70,0.46,-0.13,-0.78,0.42,0.15,0.31,0.49,0.22,-1.42,0.19,-1.02,0.38,0.63,0.26,0.01,1.20,-0.54,-0.09,0.85,0.17,0.20,-0.97,0.30,-0.19,-0.12,0.58,-0.70,0.09,0.48,-0.08,-0.71,0.01,0.48,-0.25,0.01,0.75,-0.58,-0.86,-1.62,-0.10,-0.41,-1.00,0.55,0.21,

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,Package Type,Hotel_Ratings,Start City,Cancellation Rules,Flight Stops,Meals,Journey_Day,Journey_month,Trip_Duration
0,-0.14,-0.57,0.35,0.17,0.67,-0.14,0.08,0.11,-0.53,-0.56,-0.60,1.20,-0.23,-0.22,-0.25,-0.45,0.07,1.33,-0.97,0.34,1.12,-0.22,0.08,0.18,0.78,0.45,-0.01,-0.12,0.18,-1.11,-0.37,-0.34,-0.05,-0.08,0.22,0.96,0.35,-0.47,-0.42,0.43,-0.16,0.38,-0.33,-1.44,-0.25,0.51,0.86,1.24,0.25,-0.82,0.18,-0.10,0.20,-0.74,-0.32,-0.70,-0.62,0.84,-0.52,0.23,0.45,-0.27,-0.01,-1.07,0.94,0.47,-0.37,0.05,-0.37,0.52,0.22,0.06,-0.82,0.56,-0.40,0.49,0.62,0.23,0.48,0.10,0.43,0.23,-0.37,0.07,0.18,-0.22,0.89,0.17,-0.01,-0.27,-0.93,0.80,-0.56,-0.03,-0.44,-0.50,-0.81,-0.29,-0.82,0.19,-0.05,1.60,-0.21,-0.12,0.70,-0.34,-0.09,0.47,-0.64,-0.56,0.31,-0.14,-0.60,-0.07,0.11,-0.10,-0.03,-0.62,-0.15,0.23,1.59,1.06,-0.15,-0.24,0.31,-0.32,-0.53,-0.03,1.37,0.03,0.77,-0.58,0.31,-0.45,-0.45,-0.32,0.19,0.25,0.06,0.00,1.13,1.00,-0.41,0.28,0.71,0.62,-0.41,0.46,0.49,-0.34,0.30,-0.69,0.82,0.04,-0.15,0.25,-0.44,0.43,0.51,0.35,0.32,-0.31,-0.00,-1.02,-0.34,-0.22,0.22,0.61,0.06,-0.22,-0.49,-0.20,-0.58,0.59,0.12,0.20,-1.36,1.00,-0.59,-1.16,-0.08,0.46,0.24,-0.72,1.35,-0.22,0.48,0.96,0.01,0.10,0.32,0.14,-0.79,0.94,-1.01,0.12,-0.90,0.26,0.03,-0.44,-0.11,-0.35,-0.29,0.15,-0.89,-0.92,0.67,0.19,-0.28,0.41,-0.32,-0.07,0.47,0.32,0.26,0.20,-0.85,-0.11,0.03,-0.49,-0.11,0.17,0.09,-0.44,0.19,0.51,-1.00,-0.59,0.00,-0.26,-0.39,-0.00,0.00,-0.11,-0.31,0.86,0.87,-0.01,0.69,-0.60,0.15,1.44,-0.18,0.08,0.25,0.23,-0.74,-0.17,0.31,0.61,...,0.69,-1.19,0.42,-0.83,0.14,-0.83,1.07,0.23,-0.56,0.17,0.31,0.28,-0.46,0.10,-0.15,-0.34,0.64,-0.51,0.19,0.17,1.07,-0.14,0.75,-0.12,-0.41,0.68,1.25,-0.36,-0.03,-1.27,-0.03,0.09,0.34,0.63,-0.98,0.57,1.13,1.23,-0.14,-0.73,-1.12,-0.11,-0.42,0.25,-0.30,-0.38,-1.28,0.34,0.13,-0.64,0.46,-0.44,0.04,-0.49,0.11,0.39,-0.10,0.24,0.54,0.51,-0.25,-0.75,-0.17,-0.99,0.45,-0.22,-0.30,-0.43,0.84,0.19,1.28,1.74,-0.40,1.53,1.02,0.12,-0.92,-1.40,-0.25,-0.03,0.74,-0.78,0.45,0.64,-0.86,1.11,0.45,0.05,-0.86,-0.58,0.67,0.23,-0.57,0.51,-0.39,0.03,-0.40,-0.58,-0.52,-0.33,-1.10,-0.25,-0.40,0.16,-0.32,-1.87,-0.53,0.20,-0.60,-0.03,0.85,1.06,0.3

In [ ]:
new_train.fillna(0,inplace=True)
new_test.fillna(0,inplace=True)

## Model Training

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn import model_selection
from sklearn import metrics
from sklearn.ensemble import GradientBoostingRegressor

X = new_train.drop('Per Person Price',1)
y = new_train['Per Person Price']
Xt = new_test


SEED = 42

FOLD=10

test_preds = np.zeros(len(Xt))

# k-fold cross validation
scores=list()
kf = KFold(n_splits=FOLD, shuffle=True,random_state=SEED)

# enumerate splits

for train_idx, test_idx in tqdm(kf.split(X, y)):
    # get data
    X_train, y_train = X.iloc[train_idx, :], y[train_idx]
    X_test, y_test = X.iloc[test_idx, :], y[test_idx]
    test=Xt
    # fit model
    model = LGBMRegressor(random_state=9,n_estimators=300)
    model.fit(X_train,np.log(y_train))
    # evaluate model
    yhat = np.expm1((model.predict(X_test)))
    test_preds += np.exp((model.predict(test)))/FOLD
    acc=np.sqrt(mean_squared_log_error(y_test,yhat))
    


    
    # store score
    scores.append(acc)
    print('> ', acc)
    
# summarize model performance
# summarize model performance
mean_s= np.mean(scores)
print('Mean: %.3f' % (mean_s))

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=300, n_jobs=-1, num_leaves=31, objective=None,
              random_state=9, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

>  0.189242897081014


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=300, n_jobs=-1, num_leaves=31, objective=None,
              random_state=9, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

>  0.18768953774266053


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=300, n_jobs=-1, num_leaves=31, objective=None,
              random_state=9, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

...

>  0.18698481004470088


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=300, n_jobs=-1, num_leaves=31, objective=None,
              random_state=9, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

>  0.18943944966262674


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=300, n_jobs=-1, num_leaves=31, objective=None,
              random_state=9, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

>  0.19058563980265697

Mean: 0.189


In [ ]:
test_preds=pd.DataFrame(test_preds)
test_preds.columns=['Per Person Price']
test_preds.to_csv('Lgbm_300-4.csv',index=False)

## Create Final Submission File

In [ ]:
from google.colab import files
files.download('Lgbm_300-4.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>